<a href="https://www.kaggle.com/code/mustafadagteki/sql-golden-ages-of-the-games?scriptVersionId=154765789" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## 1. The ten best-selling video games
<p>Video games are big business: the global gaming market is projected to be worth more than $300 billion by 2027 according to <a href="https://www.mordorintelligence.com/industry-reports/global-gaming-market">Mordor Intelligence</a>. With so much money at stake, the major game publishers are hugely incentivized to create the next big hit. But are games getting better, or has the golden age of video games already passed?</p>
<p>In this project, we'll explore the top 400 best-selling video games created between 1977 and 2020. We'll compare a dataset on game sales with critic and user reviews to determine whether or not video games have improved as the gaming market has grown.</p>
<p>Our database contains two tables. We've limited each table to 400 rows for this project, but you can find the complete dataset with over 13,000 games on <a href="https://www.kaggle.com/holmjason2/videogamedata">Kaggle</a>. </p>
<h3 id="game_sales"><code>game_sales</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Name of the video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>platform</code></td>
<td>varchar</td>
<td>Gaming platform</td>
</tr>
<tr>
<td style="text-align:left;"><code>publisher</code></td>
<td>varchar</td>
<td>Game publisher</td>
</tr>
<tr>
<td style="text-align:left;"><code>developer</code></td>
<td>varchar</td>
<td>Game developer</td>
</tr>
<tr>
<td style="text-align:left;"><code>games_sold</code></td>
<td>float</td>
<td>Number of copies sold (millions)</td>
</tr>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Release year</td>
</tr>
</tbody>
</table>
<h3 id="reviews"><code>reviews</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Name of the video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>critic_score</code></td>
<td>float</td>
<td>Critic score according to Metacritic</td>
</tr>
<tr>
<td style="text-align:left;"><code>user_score</code></td>
<td>float</td>
<td>User score according to Metacritic</td>
</tr>
</tbody>
</table>
<p>Let's begin by looking at some of the top selling video games of all time!</p>

In [1]:
import pandas as pd
from pandasql import sqldf

In [2]:
# Load data from CSV
game_sales = pd.read_csv('/kaggle/input/portfolio-projects-data/game_sales_data.csv', encoding='latin1', nrows=400)
reviews = pd.read_csv('/kaggle/input/portfolio-projects-data/game_reviews_data.csv', encoding='latin1', nrows=400)

In [3]:
# Create a SQL query function
psql = lambda q: sqldf(q, globals())

In [4]:
# Select all information for the top ten best-selling games
# Order the results from best-selling game down to tenth best-selling
query1 = """SELECT *
        FROM game_sales
        ORDER BY games_sold DESC
        LIMIT 10;"""
        
result1 = psql(query1)
result1

,game,platform,publisher,developer,games_sold,year
0,Wii Sports,Wii,Nintendo,Nintendo EAD,82.90,2006
1,Super Mario Bros.,NES,Nintendo,Nintendo EAD,40.24,1985
2,Counter-Strike: Global Offensive,PC,Valve,Valve Corporation,40.00,2012
3,Mario Kart Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
4,PLAYERUNKNOWN'S BATTLEGROUNDS,PC,PUBG Corporation,PUBG Corporation,36.60,2017
5,Minecraft,PC,Mojang,Mojang AB,33.15,2010
6,Wii Sports Resort,Wii,Nintendo,Nintendo EAD,33.13,2009
7,Pokemon Red / Green / Blue Version,GB,Nintendo,Game Freak,31.38,1998
8,New Super Mario Bros.,DS,Nintendo,Nintendo EAD,30.80,2006
9,New Super Mario Bros. Wii,Wii,Nintendo,Nintendo EAD,30.30,2009


## 2. Missing review scores
<p>Wow, the best-selling video games were released between 1985 to 2017! That's quite a range; we'll have to use data from the <code>reviews</code> table to gain more insight on the best years for video games. </p>
<p>First, it's important to explore the limitations of our database. One big shortcoming is that there is not any <code>reviews</code> data for some of the games on the <code>game_sales</code> table. </p>

In [5]:
# Join games_sales and reviews
# Select a count of the number of games where both critic_score and user_score are null

query2 = """SELECT 
            COUNT(g.game)
            FROM game_sales AS g
            LEFT JOIN reviews AS r 
               ON g.game = r.game
            WHERE critic_score IS NULL 
               AND user_score IS NULL;"""
    
result2 = psql(query2)
result2

,COUNT(g.game)
0,42


## 3. Years that video game critics loved
<p>It looks like a little less than ten percent of the games on the <code>game_sales</code> table don't have any reviews data. That's a small enough percentage that we can continue our exploration, but the missing reviews data is a good thing to keep in mind as we move on to evaluating results from more sophisticated queries. </p>
<p>There are lots of ways to measure the best years for video games! Let's start with what the critics think. </p>

In [6]:
# Select release year and average critic score for each year, rounded and aliased
# Join the game_sales and reviews tables
# Group by release year
# Order the data from highest to lowest avg_critic_score and limit to 10 results

query3 = """SELECT
                year,
                ROUND(AVG(critic_score),2) AS avg_critic_score
            FROM game_sales AS g
            JOIN reviews AS r
                ON g.game = r.game
            GROUP BY year
            ORDER BY avg_critic_score DESC
            LIMIT 10;"""

result3 = psql(query3)
result3

,year,avg_critic_score
0,1990,9.80
1,1992,9.67
2,2020,9.20
3,1998,9.17
4,1993,9.10
5,1995,9.07
6,2004,9.03
7,1982,9.00
8,2002,8.99
9,1999,8.93


## 4. Was 1982 really that great?
<p>The range of great years according to critic reviews goes from 1982 until 2020: we are no closer to finding the golden age of video games! </p>
<p>Hang on, though. Some of those <code>avg_critic_score</code> values look like suspiciously round numbers for averages. The value for 1982 looks especially fishy. Maybe there weren't a lot of video games in our dataset that were released in certain years. </p>
<p>Let's update our query and find out whether 1982 really was such a great year for video games.</p>

In [7]:
# Paste your query from the previous task; update it to add a count of games released in each year called num_games
# Update the query so that it only returns years that have more than four reviewed games

query4 = """SELECT
                year,
                ROUND(AVG(critic_score),2) AS avg_critic_score,
                COUNT(g.game) AS num_games
            FROM game_sales AS g
            JOIN reviews AS r
                ON g.game = r.game
            GROUP BY year
            HAVING COUNT(g.game) > 4
            ORDER BY avg_critic_score DESC
            LIMIT 10;"""

result4 = psql(query4)
result4

,year,avg_critic_score,num_games
0,1990,9.80,5
1,1992,9.67,6
2,1998,9.17,11
3,2004,9.03,11
4,2002,8.99,9
5,1999,8.93,11
6,2011,8.86,43
7,2014,8.83,58
8,2001,8.82,14
9,2018,8.74,17


## 5. Years that dropped off the critics' favorites list
<p>That looks better! The <code>num_games</code> column convinces us that our new list of the critics' top games reflects years that had quite a few well-reviewed games rather than just one or two hits. But which years dropped off the list due to having four or fewer reviewed games? Let's identify them so that someday we can track down more game reviews for those years and determine whether they might rightfully be considered as excellent years for video game releases!</p>
<p>It's time to brush off your set theory skills. To get started, we've created tables with the results of our previous two queries:</p>
<h3 id="top_critic_years"><code>top_critic_years</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Average of all critic scores for games released in that year</td>
</tr>
</tbody>
</table>
<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Count of the number of video games released in that year</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Average of all critic scores for games released in that year</td>
</tr>
</tbody>
</table>

In [8]:
# Compute the new data frames

query3a = """SELECT
                year,
                ROUND(AVG(critic_score),2) AS avg_critic_score
            FROM game_sales AS g
            JOIN reviews AS r
                ON g.game = r.game
            GROUP BY year
            ORDER BY avg_critic_score DESC
            LIMIT 10;"""

result3a= psql(query3a)

query4a = """SELECT
                year,
                ROUND(AVG(critic_score),2) AS avg_critic_score,
                COUNT(g.game) AS num_games
            FROM game_sales AS g
            JOIN reviews AS r
                ON g.game = r.game
            GROUP BY year
            HAVING COUNT(g.game) > 4
            ORDER BY avg_critic_score DESC
            LIMIT 10;"""

result4a = psql(query4a)

In [9]:
top_critic_years = pd.DataFrame(result3a)
top_critic_years_more_than_four_games = pd.DataFrame(result4a)

In [10]:
# Select the year and avg_critic_score for those years that dropped off the list of critic favorites 
# Order the results from highest to lowest avg_critic_score

query5 = """SELECT 
                year,
                avg_critic_score
            FROM top_critic_years
            EXCEPT
            SELECT
                year,
                avg_critic_score
            FROM top_critic_years_more_than_four_games
            ORDER BY avg_critic_score DESC;"""

result5 = psql(query5)
result5

,year,avg_critic_score
0,2020,9.20
1,1993,9.10
2,1995,9.07
3,1982,9.00


## 6. Years video game players loved
<p>Based on our work in the task above, it looks like the early 1990s might merit consideration as the golden age of video games based on <code>critic_score</code> alone, but we'd need to gather more games and reviews data to do further analysis. </p>
<p>Let's move on to looking at the opinions of another important group of people: players! To begin, let's create a query very similar to the one we used in Task Four, except this one will look at <code>user_score</code> averages by year rather than <code>critic_score</code> averages.</p>

In [11]:
# Select year, an average of user_score, and a count of games released in a given year, aliased and rounded
# Include only years with more than four reviewed games; group data by year
# Order data by avg_user_score, and limit to ten results

query6 = """SELECT
                year,
                ROUND(AVG(user_score),2) AS avg_user_score,
                COUNT(g.game) AS num_games
            FROM game_sales AS g
            JOIN reviews AS r
                ON g.game = r.game
            GROUP BY year
            HAVING COUNT(g.game) > 4
            ORDER BY avg_user_score DESC
            LIMIT 10;"""

result6 = psql(query6)
result6

,year,avg_user_score,num_games
0,1997,9.50,10
1,1998,9.43,11
2,1990,9.30,5
3,2010,9.24,34
4,2009,9.16,25
5,1996,9.00,5
6,2006,8.95,17
7,2005,8.95,13
8,2008,8.91,26
9,2002,8.80,9


## 7. Years that both players and critics loved
<p>Alright, we've got a list of the top ten years according to both critic reviews and user reviews. Are there any years that showed up on both tables? If so, those years would certainly be excellent ones!</p>
<p>Recall that we have access to the <code>top_critic_years_more_than_four_games</code> table, which stores the results of our top critic years query from Task 4:</p>
<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Count of the number of video games released in that year</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Average of all critic scores for games released in that year</td>
</tr>
</tbody>
</table>
<p>We've also saved the results of our top user years query from the previous task into a table:</p>
<h3 id="top_user_years_more_than_four_games"><code>top_user_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Count of the number of video games released in that year</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_user_score</code></td>
<td>float</td>
<td>Average of all user scores for games released in that year</td>
</tr>
</tbody>
</table>

In [12]:
query6a = """SELECT
                year,
                ROUND(AVG(user_score),2) AS avg_user_score,
                COUNT(g.game) AS num_games
            FROM game_sales AS g
            JOIN reviews AS r
                ON g.game = r.game
            GROUP BY year
            HAVING COUNT(g.game) > 4
            ORDER BY avg_user_score DESC
            LIMIT 10;"""

result6a = psql(query6a)

top_user_years_more_than_four_games = pd.DataFrame(result6a)

In [13]:
# Select the year results that appear on both tables

query7= """SELECT year
           FROM top_critic_years_more_than_four_games
           INTERSECT
           SELECT year
           FROM top_user_years_more_than_four_games;"""

result7 = psql(query7)
result7

,year
0,1990
1,1998
2,2002


## 8. Sales in the best video game years
<p>Looks like we've got three years that both users and critics agreed were in the top ten! There are many other ways of measuring what the best years for video games are, but let's stick with these years for now. We know that critics and players liked these years, but what about video game makers? Were sales good? Let's find out.</p>
<p>This time, we haven't saved the results from the previous task in a table for you. Instead, we'll use the query from the previous task as a subquery in this one! This is a great skill to have, as we don't always have write permissions on the database we are querying.</p>

In [14]:
# Select year and sum of games_sold, aliased as total_games_sold; order results by total_games_sold descending
# Filter game_sales based on whether each year is in the list returned in the previous task

query8 = """SELECT
                year,
                SUM(games_sold) AS total_games_sold
            FROM game_sales
            WHERE year IN (
                SELECT year
                FROM top_critic_years_more_than_four_games
                INTERSECT
                SELECT year
                FROM top_user_years_more_than_four_games)
            GROUP BY year
            ORDER BY total_games_sold DESC;"""

result8 = psql(query8)
result8

,year,total_games_sold
0,1998,101.52
1,2002,58.67
2,1990,27.47
